## Instituto Federal de Minas Gerais - Campus Bambuí 
### *Engenharia de Computação*

***Alunos: Gabriel Henrique Silva Duque e*** 
***Rafael Gonçalves Oliveira***

In [4]:
import math
import logging
import struct

# --- Constantes para estimativa de tamanho (simulação em bytes) ---
INT_SIZE = 4      # Assumindo inteiros de 4 bytes
POINTER_SIZE = 4  # Assumindo ponteiros de 4 bytes

# --- Nó/Página da Árvore ---
class BPlusNode():
    """Representa um nó interno ou uma folha na B+ Tree."""
    def __init__(self, is_leaf=False, max_keys=0, min_keys=0):
        self.keys = []
        self.children = []  # Registros (se folha) ou Nós filhos (se interno)
        self.is_leaf = is_leaf
        self.next_leaf = None
        self.parent = None
        
        # Limites definidos dinamicamente baseados no tamanho da página
        self.max_keys = max_keys
        self.min_keys = min_keys

    def is_full(self):
        return len(self.keys) > self.max_keys

    def is_underflow(self):
        return len(self.keys) < self.min_keys

    def __repr__(self):
        return f"Keys: {self.keys}, Leaf: {self.is_leaf}"

class BPlusTree():
    """Implementação da Estrutura de Índice B+ Tree configurável."""
    
    def __init__(self, num_fields, page_size, log_filename='bplustree.log', debugging=False):
        """
        Inicializa a árvore com configurações de página e registro.
        
        Args:
            num_fields (int): Número de campos inteiros em cada registro.
            page_size (int): Tamanho da página em bytes.
        """
        self.num_fields = num_fields
        self.page_size = page_size
        self.root = None # Será inicializado nos cálculos abaixo
        self._debugging = debugging
        self._config_log(log_filename)
        
        # --- Cálculos de Capacidade e Ordem ---
        # Tamanho do Registro = num_fields * 4 bytes
        self.record_size = num_fields * INT_SIZE
        self.key_size = INT_SIZE # Chave é o primeiro campo (int)

        # 1. Capacidade do Nó Folha (Armazena Registros)
        # Fórmula: L * (KeySize + RecordSize) + NextPtrSize <= PageSize
        # Onde L é o número de registros.
        # Nota: Consideramos KeySize + RecordSize como o custo de uma entrada na folha
        entry_size_leaf = self.key_size + self.record_size
        self.leaf_capacity = (page_size - POINTER_SIZE) // entry_size_leaf
        self.leaf_max_keys = self.leaf_capacity
        self.leaf_min_keys = math.ceil(self.leaf_capacity / 2)

        # 2. Capacidade do Nó Interno (Armazena Chaves e Ponteiros)
        # Fórmula: m * PointerSize + (m-1) * KeySize <= PageSize
        # m * (P + K) - K <= PageSize
        # m <= (PageSize + K) / (P + K)
        # Onde m é a Ordem (número máximo de filhos)
        self.internal_order = (page_size + self.key_size) // (POINTER_SIZE + self.key_size)
        self.internal_max_keys = self.internal_order - 1 # Num chaves é ordem - 1
        self.internal_min_keys = math.ceil(self.internal_order / 2) - 1

        # Inicializa raiz (como folha inicialmente)
        self.root = BPlusNode(is_leaf=True, max_keys=self.leaf_max_keys, min_keys=self.leaf_min_keys)

        self._debug('--- B+ Tree Criada ---')
        self._debug(f'Config: PageSize={page_size}B, NumFields={num_fields}')
        self._debug(f'Leaf Node: Max Recs={self.leaf_max_keys}, Min Recs={self.leaf_min_keys}')
        self._debug(f'Internal Node: Order={self.internal_order}, Max Keys={self.internal_max_keys}')


    # ----------------------------------------
    # I. INSERÇÃO
    # ----------------------------------------

    def insert(self, record):
        """Insere um registro (tupla de inteiros)."""
        # Validações
        if len(record) != self.num_fields:
            raise ValueError(f"Registro deve ter {self.num_fields} campos.")
        if not all(isinstance(x, int) for x in record):
            raise TypeError("Todos os campos devem ser inteiros.")

        key = record[0]
        self._debug(f"-> Inserindo chave: {key}")
        
        leaf = self._find_leaf(key)
        self._insert_into_leaf(leaf, key, record)

        if leaf.is_full():
            key_up, new_node = self._split(leaf)
            
            if leaf is self.root:
                new_root = BPlusNode(is_leaf=False, max_keys=self.internal_max_keys, min_keys=self.internal_min_keys)
                new_root.keys = [key_up]
                new_root.children = [leaf, new_node]
                leaf.parent = new_root
                new_node.parent = new_root
                self.root = new_root
            else:
                self._insert_into_parent(leaf.parent, key_up, new_node)
        
        self._debug(f"-> Inserção de {key} finalizada.")

    def _insert_into_leaf(self, leaf, key, record):
        temp_items = sorted([(k, r) for k, r in zip(leaf.keys, leaf.children)] + [(key, record)])
        leaf.keys = [item[0] for item in temp_items]
        leaf.children = [item[1] for item in temp_items]

    def _split(self, node):
        mid_point = len(node.keys) // 2
        
        # Cria novo nó herdando o tipo e os limites do nó original
        new_node = BPlusNode(is_leaf=node.is_leaf, max_keys=node.max_keys, min_keys=node.min_keys)
        new_node.parent = node.parent

        key_up = node.keys[mid_point]

        if node.is_leaf:
            new_node.keys = node.keys[mid_point:]
            new_node.children = node.children[mid_point:]
            node.keys = node.keys[:mid_point]
            node.children = node.children[:mid_point]
            
            node.next_leaf, new_node.next_leaf = new_node, node.next_leaf
            key_up = new_node.keys[0] # Cópia da chave para cima
        else:
            # Nó Interno: A chave do meio sobe e NÃO fica no nó filho
            new_node.keys = node.keys[mid_point + 1:]
            new_node.children = node.children[mid_point + 1:]
            node.keys = node.keys[:mid_point]
            
            children_moving = node.children[mid_point + 1:]
            node.children = node.children[:mid_point + 1]
            
            for child in new_node.children:
                child.parent = new_node

        return key_up, new_node

    def _insert_into_parent(self, parent, key_up, child_node):
        if parent.is_full():
            parent_key_up, new_parent = self._split(parent)
            
            if key_up < parent_key_up:
                target_node = parent
            else:
                target_node = new_parent
            
            self._insert_into_parent_simple(target_node, key_up, child_node)
            
            if parent is self.root:
                new_root = BPlusNode(is_leaf=False, max_keys=self.internal_max_keys, min_keys=self.internal_min_keys)
                new_root.keys = [parent_key_up]
                new_root.children = [parent, new_parent]
                parent.parent = new_root
                new_parent.parent = new_root
                self.root = new_root
            else:
                self._insert_into_parent(parent.parent, parent_key_up, new_parent)
        else:
            self._insert_into_parent_simple(parent, key_up, child_node)

    def _insert_into_parent_simple(self, parent, key, child):
        idx = 0
        while idx < len(parent.keys) and key > parent.keys[idx]:
            idx += 1
        parent.keys.insert(idx, key)
        parent.children.insert(idx + 1, child)
        child.parent = parent

    # ----------------------------------------
    # II. REMOÇÃO
    # ----------------------------------------

    def remove(self, key):
        self._debug(f"-> Tentando remover chave: {key}")
        leaf = self._find_leaf(key)
        
        if key not in leaf.keys:
            self._debug("Chave não encontrada.")
            return

        idx = leaf.keys.index(key)
        leaf.keys.pop(idx)
        leaf.children.pop(idx)

        if leaf == self.root:
            if len(leaf.keys) == 0:
                self.root = BPlusNode(is_leaf=True, max_keys=self.leaf_max_keys, min_keys=self.leaf_min_keys)
        elif leaf.is_underflow():
            self._delete_entry(leaf)
        
        self._debug(f"-> Remoção de {key} finalizada.")

    def _delete_entry(self, node):
        if node == self.root:
            if len(node.keys) == 0 and len(node.children) > 0:
                self.root = node.children[0]
                self.root.parent = None
            return

        parent = node.parent
        idx = parent.children.index(node)
        
        sibling = None
        is_left_sibling = False
        
        if idx > 0:
            sibling = parent.children[idx - 1]
            is_left_sibling = True
        elif idx < len(parent.children) - 1:
            sibling = parent.children[idx + 1]
            is_left_sibling = False
        
        if not sibling:
            return 

        # Verifica se cabe fusão (soma chaves <= maximo do nó atual)
        if len(node.keys) + len(sibling.keys) <= node.max_keys:
            self._merge(node, sibling, parent, idx, is_left_sibling)
        else:
            self._redistribute(node, sibling, parent, idx, is_left_sibling)

    def _merge(self, node, sibling, parent, idx, is_left_sibling):
        self._debug(f"--- Fazendo MERGE ---")
        
        if is_left_sibling:
            left_node, right_node = sibling, node
            split_key_idx = idx - 1
        else:
            left_node, right_node = node, sibling
            split_key_idx = idx

        separator_key = parent.keys[split_key_idx]

        if node.is_leaf:
            left_node.keys.extend(right_node.keys)
            left_node.children.extend(right_node.children)
            left_node.next_leaf = right_node.next_leaf
        else:
            left_node.keys.append(separator_key)
            left_node.keys.extend(right_node.keys)
            left_node.children.extend(right_node.children)
            for child in right_node.children:
                child.parent = left_node

        parent.keys.pop(split_key_idx)
        parent.children.pop(split_key_idx + 1) 

        if parent.is_underflow():
            self._delete_entry(parent)

    def _redistribute(self, node, sibling, parent, idx, is_left_sibling):
        self._debug(f"--- Fazendo EMPRÉSTIMO (Redistribution) ---")
        
        if node.is_leaf:
            if is_left_sibling:
                borrowed_key = sibling.keys.pop()
                borrowed_val = sibling.children.pop()
                node.keys.insert(0, borrowed_key)
                node.children.insert(0, borrowed_val)
                parent.keys[idx - 1] = node.keys[0]
            else:
                borrowed_key = sibling.keys.pop(0)
                borrowed_val = sibling.children.pop(0)
                node.keys.append(borrowed_key)
                node.children.append(borrowed_val)
                parent.keys[idx] = sibling.keys[0]
        else:
            if is_left_sibling:
                separator_idx = idx - 1
                separator_key = parent.keys[separator_idx]
                
                child_to_move = sibling.children.pop()
                key_to_move = sibling.keys.pop()
                
                node.keys.insert(0, separator_key)
                node.children.insert(0, child_to_move)
                child_to_move.parent = node
                
                parent.keys[separator_idx] = key_to_move
            else:
                separator_idx = idx
                separator_key = parent.keys[separator_idx]
                
                child_to_move = sibling.children.pop(0)
                key_to_move = sibling.keys.pop(0)
                
                node.keys.append(separator_key)
                node.children.append(child_to_move)
                child_to_move.parent = node
                
                parent.keys[separator_idx] = key_to_move

    # ----------------------------------------
    # III. BUSCA E UTILITÁRIOS
    # ----------------------------------------

    def search(self, key):
        """Busca por igualdade: Retorna lista de registros com a chave."""
        leaf = self._find_leaf(key)
        found_records = []
        for i, k in enumerate(leaf.keys):
            if k == key:
                found_records.append(leaf.children[i])
            elif k > key:
                break
        return found_records

    def range_search(self, start_key, end_key):
        """Busca por intervalo: Retorna registros onde start_key <= chave <= end_key."""
        self._debug(f"-> Buscando intervalo: [{start_key}, {end_key}]")
        results = []
        
        # 1. Encontrar o nó folha onde o intervalo começa
        current_node = self._find_leaf(start_key)
        
        # 2. Percorrer as folhas sequencialmente usando next_leaf
        while current_node is not None:
            for i, key in enumerate(current_node.keys):
                if key >= start_key:
                    if key <= end_key:
                        results.append(current_node.children[i])
                    else:
                        # Se a chave atual já passou do fim do intervalo, podemos parar
                        return results
            
            # Se a maior chave dessa folha ainda é menor que end_key, vamos para a próxima folha
            if current_node.keys[-1] <= end_key:
                 current_node = current_node.next_leaf
            else:
                # Caso contrário, o intervalo terminou dentro desta página
                break
                
        return results

    def _find_leaf(self, key):
        node = self.root
        while not node.is_leaf:
            child_index = 0
            while child_index < len(node.keys) and key >= node.keys[child_index]:
                child_index += 1
            node = node.children[child_index]
        return node

    def _config_log(self, log):
        self._log = logging.getLogger(log)
        str_format = '%(levelname)s - %(message)s'
        logging.basicConfig(level=logging.INFO, format=str_format) 
        if self._debugging:
            self._log.setLevel(logging.DEBUG)

    def _debug(self, msg, *args, **kwargs):
        if self._debugging:
            self._log.debug(msg, *args, **kwargs)

    def __repr__(self):
        def print_node(node, level=0):
            indent = "  " * level
            output = f"\n{indent}- Lvl {level} [{'Leaf' if node.is_leaf else 'Node'}]: {node.keys}"
            if not node.is_leaf:
                for child in node.children:
                    output += print_node(child, level + 1)
            return output
        return f"--- B+ Tree (Page: {self.page_size}B, Fields: {self.num_fields}) ---" + print_node(self.root)

# --- Teste Final com Configuração ---
if __name__ == '__main__':
    print("=== INICIANDO TESTE COMPLETO ===")
    tree = BPlusTree(num_fields=3, page_size=64, debugging=True)

    data = [
        (10, 100, 200), (20, 101, 201), (5, 102, 202), 
        (6, 103, 203), (12, 104, 204), (30, 105, 205), 
        (7, 106, 206), (17, 107, 207)
    ]
    
    print("\n=== INSERINDO DADOS ===")
    for item in data:
        tree.insert(item)
    print(tree)

    print("\n=== BUSCA POR IGUALDADE (Chave 12) ===")
    print(tree.search(12))

    print("\n=== BUSCA POR INTERVALO (Chaves 6 a 18) ===")
    # Deve retornar: 6, 7, 10, 12, 17
    results = tree.range_search(6, 18)
    print(f"Registros encontrados: {len(results)}")
    for r in results:
        print(r)

    print("\n=== REMOÇÃO (Chave 6) ===")
    tree.remove(6)
    print(tree)

DEBUG - --- B+ Tree Criada ---
DEBUG - --- B+ Tree Criada ---
DEBUG - Config: PageSize=64B, NumFields=3
DEBUG - Config: PageSize=64B, NumFields=3
DEBUG - Leaf Node: Max Recs=3, Min Recs=2
DEBUG - Leaf Node: Max Recs=3, Min Recs=2
DEBUG - Internal Node: Order=8, Max Keys=7
DEBUG - Internal Node: Order=8, Max Keys=7
DEBUG - -> Inserindo chave: 10
DEBUG - -> Inserindo chave: 10
DEBUG - -> Inserção de 10 finalizada.
DEBUG - -> Inserção de 10 finalizada.
DEBUG - -> Inserindo chave: 20
DEBUG - -> Inserindo chave: 20
DEBUG - -> Inserção de 20 finalizada.
DEBUG - -> Inserção de 20 finalizada.
DEBUG - -> Inserindo chave: 5
DEBUG - -> Inserindo chave: 5
DEBUG - -> Inserção de 5 finalizada.
DEBUG - -> Inserção de 5 finalizada.
DEBUG - -> Inserindo chave: 6
DEBUG - -> Inserindo chave: 6
DEBUG - -> Inserção de 6 finalizada.
DEBUG - -> Inserção de 6 finalizada.
DEBUG - -> Inserindo chave: 12
DEBUG - -> Inserindo chave: 12
DEBUG - -> Inserção de 12 finalizada.
DEBUG - -> Inserção de 12 finalizada.
DE

=== INICIANDO TESTE COMPLETO ===

=== INSERINDO DADOS ===
--- B+ Tree (Page: 64B, Fields: 3) ---
- Lvl 0 [Node]: [10, 20]
  - Lvl 1 [Leaf]: [5, 6, 7]
  - Lvl 1 [Leaf]: [10, 12, 17]
  - Lvl 1 [Leaf]: [20, 30]

=== BUSCA POR IGUALDADE (Chave 12) ===
[(12, 104, 204)]

=== BUSCA POR INTERVALO (Chaves 6 a 18) ===
Registros encontrados: 5
(6, 103, 203)
(7, 106, 206)
(10, 100, 200)
(12, 104, 204)
(17, 107, 207)

=== REMOÇÃO (Chave 6) ===
--- B+ Tree (Page: 64B, Fields: 3) ---
- Lvl 0 [Node]: [10, 20]
  - Lvl 1 [Leaf]: [5, 7]
  - Lvl 1 [Leaf]: [10, 12, 17]
  - Lvl 1 [Leaf]: [20, 30]
